# Validate Downscale Skill vs. ERA5

This example shows how to evaluate Salient's daily downscaled forecasts and calculate meaningful metrics. It demonstrates [validation best practices](https://salientpredictions.notion.site/Validation-0220c48b9460429fa86f577914ea5248) such as:

- Proper scoring using the Ensemble Continuous Ranked Probability Score (CRPS)
  - Considers the full forecast distribution to reward both accuracy and precision
  - Less sensitive to climatology decisions than metrics like Anomaly Correlation
- A long backtesting period (2015-2022)
  - Short evaluation periods are subject to noise


In [ ]:
# Initialize the environment:
import os
import sys

import numpy as np
import pandas as pd

# import xarray as xr
# import matplotlib.pyplot as plt

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

sk.set_file_destination("validate_ensemble_example")
sk.login("username", "password")

## Customize The Validation

This notebook is written flexibly so you have the option of validating Salient and other forecasts multiple ways. These variables will control what, when, and how the validation proceeds.

The `split_set` variable controls the amount of data to request via the `start_date` and `end_date` variables.

- `sample` - a single season of data, good for quickly making sure that the mechanics of the process work.
- `test` - gets data from 2015-2022, which is completely out-of-sample from model training. This requests a medium amount of data and is recommended for most validation processes.
- `all` - gets data from 2000-2022, representing the full historical evaluation record. This will download quite a bit of data and is not recommended for most applications.


In [ ]:
# 1. Set the meteorological variable that we'll be evaluating:
vars = ["temp", "precip"]  # wspd, tsi

freq = "daily"
# freq = "hourly"

length = 35


# 3. Set the number of hindcasts to download for validation:
split_set = "sample"  # fast demonstration of mechanics
# split_set = "test"  # recommended to validate out-of-sample with hindcast_summary
# split_set = "all"  # download every hindcast since 2000

# 4. Set the reference model to compare Salient blend to
ref_model = "clim"  # Climatology.  Works across all timescale values.
# ref_model = "noaa_gefs"  # Valid for the sub-seasonal timescale
# ref_model = "ecmwf_ens" # Valid for sub-seasonal timescale
# ref_model = "ecmwf_seas5" # Valid for seasonal and long-range timescales


# ===== Additional shared variables ==========================
force = False  # If "False", cache data calls.  Set to "True" to overwrite caches

(start_date, end_date) = {
    "sample": ("2021-04-01", "2021-07-31"),
    "test": ("2015-01-01", "2022-12-31"),
    "all": ("2000-01-01", "2022-12-31"),
}[split_set]

## Set the Area of Interest

The Salient SDK uses a "Location" object to specify the geographic bounds of a request. In this case, we will be validating against the vector of airport locations that are used to settle the Chicago Mercantile Exchange's Cooling and Heating Degree Day contracts. With `load_location_file` we can see that the file contains:

- `lat` / `lon`: latitude and longitude of the met station, standard for a `location_file`
- `name`: the 3-letter IATA airport code of the location, also `location_file` standard
- `ghcnd`: the global climate network ID of the station, used to validate against observations. To customize this analysis for any set of observation stations, use the NCEI [stations list](https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt).
- `cme`: the CME code for the location used to create CDD/HDD strip codes.
- `description`: full name of the airport

If you have a list of locations already defined in a separate CSV file, you can use [`upload_file`](https://sdk.salientpredictions.com/api/#salientsdk.upload_file) to upload the file directly without building it in code via `upload_location_file`.


In [ ]:
# fmt: off
loc = sk.Location(location_file=sk.upload_location_file(
    lats =[33.62972     ,      42.36057,      34.19966,      41.96017,      39.04443,      32.89744,      29.98438,      36.07190,      44.88523,      40.77945,      39.87326,      45.59578,      38.50659],
    lons =[-84.44224    ,     -71.00975,    -118.36543,     -87.93164,     -84.67241,     -97.02196,     -95.36072,    -115.16343,     -93.23133,     -73.88027,     -75.22681,    -122.60919,    -121.49604],
    names=["ATL"        ,         "BOS",         "BUR",         "ORD",         "CVG",         "DFW",         "IAH",         "LAS",         "MSP",         "LGA",         "PHL",         "PDX",         "SAC"],
    ghcnd=["USW00013874", "USW00014739", "USW00023152", "USW00094846", "USW00093814", "USW00003927", "USW00012960", "USW00023169", "USW00014922", "USW00014732", "USW00013739", "USW00024229", "USW00023232"],
    cme  =["1"          ,           "W",           "P",           "2",           "3",           "5",           "R",           "0",           "Q",           "4",           "6",           "7",           "S"],
    geoname="cmeus",
    force=force,
    description=["Atlanta Hartsfield", "Boston Logan", "Burbank-Glendale-Pasadena", "Chicago O'Hare", "Cincinnati (Covington)","Dallas-Fort Worth", "Houston-George Bush", "Las Vegas McCarran", "Minneapolis-StPaul", "New York La Guardia","Philadelphia", "Portland", "Sacramento Exec"],
))
# fmt: on
stations = loc.load_location_file()
print(stations)

         lat        lon name        ghcnd cme                description                     geometry
0   33.62972  -84.44224  ATL  USW00013874   1         Atlanta Hartsfield   POINT (-84.44224 33.62972)
1   42.36057  -71.00975  BOS  USW00014739   W               Boston Logan   POINT (-71.00975 42.36057)
2   34.19966 -118.36543  BUR  USW00023152   P  Burbank-Glendale-Pasadena  POINT (-118.36543 34.19966)
3   41.96017  -87.93164  ORD  USW00094846   2             Chicago O'Hare   POINT (-87.93164 41.96017)
4   39.04443  -84.67241  CVG  USW00093814   3     Cincinnati (Covington)   POINT (-84.67241 39.04443)
5   32.89744  -97.02196  DFW  USW00003927   5          Dallas-Fort Worth   POINT (-97.02196 32.89744)
6   29.98438  -95.36072  IAH  USW00012960   R        Houston-George Bush   POINT (-95.36072 29.98438)
7   36.07190 -115.16343  LAS  USW00023169   0         Las Vegas McCarran   POINT (-115.16343 36.0719)
8   44.88523  -93.23133  MSP  USW00014922   Q         Minneapolis-StPaul   POINT (

## Historical Data

To calculate forecast skill, we will want to compare forecasts made in the past with actuals. There are two flavors of actual data: (1) The ERA5 reanalysis dataset and (2) point weather station observations.

Salient's forecast natively predicts (1) ERA5, but contains a debiasing function to remove bias between ERA5 and (2) station observations.


### Historical ERA5 Data

Download daily historical values from [`data_timeseries`](https://sdk.salientpredictions.com/api/#salientsdk.data_timeseries) and then aggregate to match the forecasts, so that we can ensure that all forecasts use the same dates.

Also, get observed weather station data in the same format by downloading directly from NCEI.


In [ ]:
hist = sk.data_timeseries(
    loc=loc,
    variable=vars,
    field="vals",
    start=np.datetime64(start_date) - np.timedelta64(5, "D"),
    end=np.datetime64(end_date) + np.timedelta64(length + 1, "D"),
    frequency=freq,
    verbose=False,
    force=force,
)
print(hist)

                                           file_name variable
0  validate_ensemble_example/data_timeseries_2bcd...     temp
1  validate_ensemble_example/data_timeseries_7001...   precip


## Downscale

The [`downscale`](https://sdk.salientpredictions.com/api/#salientsdk.downscale) API endpoint and SDK function converts Salient's native weekly/monthly/quarterly probabilistic forecasts into a daily or hourly ensemble timeseries.

This is the most heavyweight call in the notebook, since it's getting multiple historical forecasts.


In [ ]:
date_range = pd.date_range(start=start_date, end=end_date, freq="MS").strftime("%Y-%m-%d").tolist()
fcst = sk.downscale(
    loc=loc,
    variables=vars,
    date=date_range,
    frequency=freq,
    length=length,
    verbose=False,
    force=force,
    strict=False,
)
# Check to see if there are any missing forecasts:
fcst_na = fcst[fcst["file_name"].isna()]
if not fcst_na.empty:
    print("Missing forecast dates:")
    print(fcst_na)
print(fcst)

                                           file_name        date
0  validate_ensemble_example/downscale_d10ed2da13...  2021-04-01
1  validate_ensemble_example/downscale_cee1ed362f...  2021-05-01
2  validate_ensemble_example/downscale_654d58423a...  2021-06-01
3  validate_ensemble_example/downscale_30c6a2c605...  2021-07-01


## Calculate Skill Metrics

Compare the forecast and ERA5 datasets to see how well they match. Here we will calculate the same "Continuous Ranked Probability Score" that resulted from the call to `hindcast_summary` earlier.


In [ ]:
skill = sk.skill.crps_ensemble(observations=hist, forecasts=fcst)
print(skill)

<xarray.Dataset> Size: 37kB
Dimensions:          (forecast_date: 4, lead: 35, location: 13)
Coordinates:
  * location         (location) object 104B 'ATL' 'BOS' 'BUR' ... 'PDX' 'SAC'
    lat              (location) float64 104B 33.63 42.36 34.2 ... 45.6 38.51
    lon              (location) float64 104B -84.44 -71.01 ... -122.6 -121.5
  * lead             (lead) timedelta64[ns] 280B 0 days 1 days ... 34 days
  * forecast_date    (forecast_date) datetime64[ns] 32B 2021-04-01 ... 2021-0...
Data variables:
    crps_temp_all    (forecast_date, lead, location) float64 15kB 0.4152 ... ...
    crps_precip_all  (forecast_date, lead, location) float64 15kB 0.4672 ... ...
    crps_temp        (lead, location) float64 4kB 0.2553 0.5017 ... 1.558 1.944
    crps_precip      (lead, location) float64 4kB 0.7846 1.082 ... 0.005594
Attributes:
    short_name:  crps
    long_name:   CRPS


In [ ]:
# skill.crps.plot.line(x="lead", hue="location", figsize=(12, 6));

# plt.xlabel("Lead Time")
# plt.ylabel("CRPS")
# plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
# plt.tight_layout()